In [15]:
using ITensors, ITensorMPS, Optim, LinearAlgebra, Printf, Random, Plots, DelimitedFiles
Random.seed!(123)
include("src/QuantumResource1D.jl")
include("src/HL_module.jl")

all_params_mat = readdlm("DATA/all_params_mat.txt") 
g_vals = vec(readdlm("DATA/g_values.txt"));

In [16]:
N = 5

λ_vals, Fidelities_HL = [], []
H_params, delta_E_HL = [], []
HL_X_exp, HL_O_exp, HL_ZZ_exp = [], [], []

for ind_g in 1:length(g_vals)
    println("Current value: g = $(g_vals[ind_g])")
    θ_g = all_params_mat[:, ind_g]

    qr_g = QR1D.QuantumResource1D(N)
    psi_g = QR1D.apply_ladder_circuit(qr_g, θ_g)

    hl_g = HL1D.HamiltonianLearner(psi_g)
    λ_min_g, v_min_g = HL1D.LearnHamiltonian_corr_mat(hl_g)

    push!(λ_vals, λ_min_g)
    push!(H_params, v_min_g)
    push!(delta_E_HL, hl_g.E_psi - hl_g.E_GS)
    push!(Fidelities_HL, hl_g.Fidelity)

    push!(HL_X_exp, real(inner(hl_g.ψ_GS, qr_g.aux_ops[1], hl_g.ψ_GS)))
    push!(HL_O_exp, real(inner(hl_g.ψ_GS, qr_g.aux_ops[2], hl_g.ψ_GS)))
    push!(HL_ZZ_exp, real(inner(hl_g.ψ_GS, qr_g.aux_ops[3], hl_g.ψ_GS)))
end

Current value: g = -1.0
Degeneracy = 10
After sweep 1 energy=-3.316624790351738  maxlinkdim=2 maxerr=2.88E-16 time=0.005
After sweep 2 energy=-3.316624790351736  maxlinkdim=2 maxerr=3.55E-16 time=0.001
After sweep 3 energy=-3.3166247903517387  maxlinkdim=2 maxerr=3.37E-16 time=0.001
After sweep 4 energy=-3.316624790351738  maxlinkdim=2 maxerr=4.20E-16 time=0.015
After sweep 5 energy=-3.3166247903517365  maxlinkdim=2 maxerr=4.94E-16 time=0.001
After sweep 6 energy=-3.316624790351736  maxlinkdim=2 maxerr=4.27E-16 time=0.001
After sweep 7 energy=-3.3166247903517374  maxlinkdim=2 maxerr=3.43E-16 time=0.002
After sweep 8 energy=-3.316624790351738  maxlinkdim=2 maxerr=3.46E-16 time=0.001
Fidelity = 1.0
GS energy = -3.316624790351738
Current value: g = -0.9797979797979798
Degeneracy = 3
After sweep 1 energy=-1.4538552402478144  maxlinkdim=2 maxerr=3.93E-12 time=0.008
After sweep 2 energy=-1.4538552402511646  maxlinkdim=2 maxerr=1.32E-11 time=0.001
After sweep 3 energy=-1.4538552402546292  max

InterruptException: InterruptException:

Plotting

In [17]:
#Verify that the state is the GS of the learned H
pl1 = plot(g_vals, delta_E_HL, label="delta E", title="Energy relative to GS g", xlabel="g", ylabel="delta E")
pl2 = plot(g_vals, λ_vals, label="λ_vals", xlabel="g", ylabel="λ")
pl3 = plot(g_vals, Fidelities_HL, label="F", xlabel="g", ylabel="F")

display(pl1)
display(pl2)
display(pl3)

attempt to save state beyond implementation limit
attempt to save state beyond implementation limit
attempt to save state beyond implementation limit
attempt to save state beyond implementation limit
attempt to save state beyond implementation limit
attempt to save state beyond implementation limit
attempt to save state beyond implementation limit
attempt to save state beyond implementation limit
attempt to save state beyond implementation limit


In [18]:
p = plot(g_vals, HL_O_exp, label="<O>",linewidth=1);
plot!(p, g_vals, HL_ZZ_exp / (N - 1), label="<ZZ>",linestyle=:dash,linewidth=3);
display(p)

attempt to save state beyond implementation limit


In [19]:
ind = 80
@show Fidelities_HL[ind]

v = H_params[ind]
bar(1:length(v), v;
    xlabel="Index",
    ylabel="learned H",
    legend=false,
    lw=2,
    ylim=(-1.1, 1.1),
    title="Hamiltonian for g = $(g_vals[ind])"
    )

BoundsError: BoundsError: attempt to access 6-element Vector{Any} at index [80]

In [20]:
#Analsysis of the learned H
param4_vals = [v[4] for v in H_params]
param9_vals = [v[9] for v in H_params]

pl1 = plot(g_vals, param4_vals, label="4", xlabel="g", ylabel="strength")
plot!(pl1, g_vals, param9_vals, label="9", xlabel="g", ylabel="strength")

attempt to save state beyond implementation limit
attempt to save state beyond implementation limit
attempt to save state beyond implementation limit
attempt to save state beyond implementation limit
attempt to save state beyond implementation limit
